In [1]:
import torch
import torch.nn as nn
import numpy as np

from torch.utils.data import Dataset, DataLoader

In [12]:
## Coding up a transformers architecture
# Embedding layer
# Positional encoding (done)
# Multi-head attention (done)
# Feedforward network (done)
# Residual connections
# Layer normalization
# Encoder
# Decoder

class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, d_model, drop_prob=0):
        super(MultiHeadAttention, self).__init__()
        assert d_model % n_heads == 0

        # Initialize dimensions
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads

        # Linear layers for transforming inputs
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(drop_prob)

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()

        # Transposing gives dimension [batch_size, n_heads, seq_length, d_k]
        return x.view(batch_size, seq_length, self.n_heads, self.d_k).transpose(1, 2)

    def attention(self, Q, K, V, mask=None):
        # Calcualte attention scores
        # [batch_size, n_heads, seq_length, d_k] * [batch_size, n_heads, d_k, seq_length]
        # =  [batch_size, n_heads, seq_length, seq_length]
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)

        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask==0, -1e9)
        
        # [batch_size, num_heads, seq_length, seq_length] 
        attn_probs = torch.softmax(attn_scores, dim=-1)

        attn_probs = self.dropout(attn_probs) # Dropout can be applied in a number of places

        # [batch_size, num_heads, seq_length, seq_length] * [batch_size, n_heads, seq_length, d_k]
        # = [batch_size, n_heads, seq_length, d_k]
        output = torch.matmal(attn_probs, V)
        return output

    def combine_heads(self, x):
        batch_size, n_heads, seq_length, d_k = x.size()

        # Recombine tensor to shape [batch_size, seq_length, d_model]
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask):

        # Apply linear transformation [batch_size, seq_length, d_model]
        Q = self.W_q(Q)
        K = self.W_k(K)
        V = self.W_v(V)

        # Split heads [batch_size, n_heads, seq_length, d_k]
        Q = self.split_heads(Q)
        K = self.split_heads(K)
        V = self.split_heads(V)

        # Perform attention on each
        attn_output = self.attention(Q, K, V, mask)

        # Recombine heads
        attn_concat = self.combine_heads(attn_output)
        
        # Output transformation
        output = self.W_o(attn_concat)
        return output

In [13]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [14]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(max_seq_length).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # [1, max_seq_length, d_model]
        pe = pe.unsqueeze(0)

        # Register as non-learnable parameter
        self.register_buffer("pe", pe)
    
    def forward(self, x):
        # Ensure the positional encodings match the actual sequence of x
        return x + self.pe[:, :x.size(1)]

In [15]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(n_heads, d_model)
        self.feedforward = FeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feedforward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [16]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(n_heads, d_model)
        self.cross_attn = MultiHeadAttention(n_heads, d_model)
        self.feedforward = FeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feedforward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, n_heads, n_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)])

        self.fc = nn.Linear(self, n_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

        
    def forward(self, src, tgt, src_mask, tgt_mask):
        src_embeddings = self.encoder_embedding(src)
        src_embeddings = self.dropout(self.positional_encoding(src_embeddings))

        tgt_embeddings = self.decoder_embedding(tgt)
        tgt_embeddings = self.dropout(self.positional_encoding(tgt_embeddings))

        enc_output = src_embeddings
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)
        
        dec_output = dec_embeddings
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)
        
        output = self.fc(dec_outputs)
        return output

In [5]:
x = torch.randn(100, 3)

y = x * np.sin(x) + 0.01

In [6]:
class ExampleDataset(Dataset):
    def __init__(self, x, y):
        super(ExampleDataset, self).__init__()
        self.x = x
        self.y = y
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.x.shape[0]

In [7]:
dataset = ExampleDataset(x, y)

In [8]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [9]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

criterion = nn.MSELoss()

epochs = 100
for epoch in range(epochs):
    for i, (x, y) in enumerate(dataloader):
        y_hat = model(x)
        loss = criterion(y_hat, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [10]:
with torch.no_grad():
    model.eval()